In [1]:
from xhtml2pdf import pisa
from xhtml2pdf.config.httpconfig import httpConfig

In [2]:
# State

In [3]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_z0_1q, get_z1_1q, get_x0_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
povms = [povm_x, povm_y, povm_z]

In [6]:
true_object = get_z0_1q(c_sys)
num_data = [100, 1000]
iteration = 100

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
]

qtomography_list = [
    StandardQst(povms, on_para_eq_constraint=True),
    StandardQst(povms, on_para_eq_constraint=False),
    StandardQst(povms, on_para_eq_constraint=True),
    StandardQst(povms, on_para_eq_constraint=False),
]
p_list = [True, False, True, False]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
]

estimation_results_list = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]

    print(f"Case {i}: {name}")
    print(f"Parametorization: {p_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
        qtomography=qtomography,
        true_object=true_object,
        num_data=num_data,
        estimator=estimator,
        iteration=iteration,
    )
    estimation_results_list.append(estimation_results)

  3%|▎         | 3/100 [00:00<00:03, 28.75it/s]

Case 0: LinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQst
Estimator: LinearEstimator


  4%|▍         | 4/100 [00:00<00:02, 38.32it/s]

Case 1: LinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQst
Estimator: LinearEstimator


  0%|          | 0/100 [00:00<?, ?it/s]/Users/tomoko/project/rcast/workspace/quara/quara/objects/state.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  np.array(vec_new, dtype=np.float64),
  1%|          | 1/100 [00:00<00:10,  9.34it/s]

Case 2: ProjectedLinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQst
Estimator: ProjectedLinearEstimator


  3%|▎         | 3/100 [00:00<00:04, 21.65it/s]

Case 3: ProjectedLinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQst
Estimator: ProjectedLinearEstimator


100%|██████████| 100/100 [00:04<00:00, 21.96it/s]


一時保存時のファイル名規則:
{ケース名}_{グラフの種類}_{numの情報}_{グラフの種類固有の識別子}

例: 
等式制約の場合:    
- case=0_trace_num=100
不等式制約の場合
- case=0_eigenvalues_num=100_i=0
- case=0_sum-unphysical-eigenvalues_num=100_greater-than-one
- case=0_sum-unphysical-eigenvalues_num=100_less-than-zero

In [7]:
tester_objects = povms

In [8]:
# MSE
iteration = 100

mse_div = report.generate_mse_div(
    estimation_results_list=estimation_results_list,
    case_name_list=case_name_list,
    true_object=true_object,
    num_data=num_data,
    n_rep=iteration,
    qtomographies=[StandardQst(tester_objects, on_para_eq_constraint=True)]
)

In [9]:
# 残すこと
case_id = 0
case_name = "LinearEstimator(True)"
target = estimation_results_list[0]

physicality_violation_test_div_true = f"""
<h2>on_para_eq_constraint=True</h2>
<h3>Case {case_id}: {case_name}<h3>
{report.generate_trace_div(target, case_id=case_id, num_data=num_data)}
"""

# Falseの場合
case_id = 1
case_name = "LinearEstimator(False)"
target = estimation_results_list[1]

physicality_violation_test_div_false = f"""
<h2>on_para_eq_constraint=False</h2>
<h3>Case {case_id}: {case_name}<h3>
<h4>Eigenvalue</h4>
{report.generate_eigenvalues_div(target, case_id=case_id, num_data=num_data, true_object=true_object)}
<h4>Sum of unphysical eigenvalues </h4>
{report.generate_sum_eigenvalues_div(target, case_id=case_id, num_data=num_data, true_object=true_object)}
"""

physicality_violation_test_div = (
    physicality_violation_test_div_true + physicality_violation_test_div_false
)

100%|██████████| 100/100 [00:00<00:00, 5512.95it/s]


In [73]:
# True Object and Tester Objects
true_object_table = report._convert_object_to_datafrane(true_object).to_html(
    classes="true_object_table", escape=False, header=False
)
tester_table = report._convert_objects_to_multiindex_dataframe(tester_objects).to_html(
    classes="tester_objects_table", escape=False, header=False
)

In [74]:
# Experiment Condition
type_tomography_values = list(set([qt.__class__.__name__ for qt in qtomography_list]))

info = {
    "Type of tomography": type_tomography_values,
    "Nrep": [iteration],
    "N": [num_data],
}
condition_df = pd.DataFrame(info).T
condition_table = condition_df.to_html(
    classes="condition_table", escape=False, header=False
)

In [75]:
# Cases
case_dict = dict(
    Name=case_name_list,
    Parameterization=p_list,
    Tomography=[t.__class__.__name__ for t in qtomography_list],
    Estimator=[e.__class__.__name__ for e in estimator_list],
)

df = pd.DataFrame(case_dict)

# case_table = df.to_html('class="case_table"', escape=False)

styles = [
    dict(selector=".col0", props=[("width", "400px")]),
    dict(selector=".col1", props=[("width", "200px")]),
    dict(selector=".col2", props=[("width", "200px")])
]
case_table = df.style.set_table_styles(styles).render()


In [89]:
report_html = f"""<html>
<head>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    
    <style type="text/css">
        <!--
             body {{color: #666666;}}
             h1 {{margin-top: 60px; border-top: 2px #dcdcdc solid; padding-top: 10px; font-size: 25px}}
             h2 {{font-size: 20px}}
             h3 {{color:#EB9348; font-size: 15px;}}
             h4{{font-size: 15px; color: #618CBC; }}
            {report._inline_block_css}
            {report._table_css}
            {report._table_contents_css}
         -->
    </style>
<title>タイトル</title>
</head>
<body>
<div id="table_of_contents">
<h1>Table of contents</h1>
<pdf:toc />
</div>
<h1>Experimental condition</h1>
<div>
{condition_table}
</div>
<h2>True object</h2>
<div>
{true_object_table}
</div>
<h2>Tester objects</h2>
<div>
{tester_table}
<h2>Cases</h2>
<div>
{case_table}
<h1>Consistency test</h1>
<div>(TODO)</div>
</div><h1>MSE</h1>
<div>
{mse_div}
</div>
<h1>Physicality violation test</h1>
{physicality_violation_test_div}

<div>
end
</div>
</body>
</html>"""

In [90]:
with open("test.html", "w") as f:
    f.write(report_html)
    
report._convert_html2pdf(report_html, "sample_1.pdf")

0

In [91]:
!open test.html

In [92]:
!open sample_1.pdf